In [38]:
from tqdm import tqdm
tqdm.pandas()
from glob import glob

import json
import csv
import numpy as np
import pandas as pd
from collections import Counter

import rdflib
from rdflib import Graph
from data.data import CollectionAccessor, ImageHandler, get_latest

from search import Search, Randomiser

/home/valentin/home2-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [39]:

# dmg_meta = dict(name="DMG 2025-04-03", id_="DMG_2025-04-03",
#                 creation_timestamp="2025-04-03")
# # dmg = CollectionAccessor.get_DMG("./data/DMG_2025-03-26.csv", "", **dmg_meta)
# image_folder = "./data/images/DMG"
# img = ImageHandler(image_folder=image_folder)

# dmg = CollectionAccessor.get_DMG(pub_path="./data/dumps/API_dump_public_2025-04-03_extracted.csv",
#                                  priv_path="./data/dumps/API_dump_private_2025-04-03_extracted.csv",
#                                  rights_path="./data/rights.csv",
#                                  image_handler=img,
#                                  **dmg_meta)

In [40]:
def init_DMG():
    image_folder = "./data/images/DMG"
    image_handler = ImageHandler(image_folder=image_folder)

    dmg_meta = dict(name="Design Museum Gent (public & private)", id_="DMG_2025-05-06",
                creation_timestamp="2025-05-06")
    df = CollectionAccessor.get_DMG(pub_path=get_latest("./data/dumps", contains="public"),
                                     priv_path=get_latest("./data/dumps", contains="private"),
                                     rights_path="./data/rights.csv",
                                     image_handler=image_handler,
                                     **dmg_meta)
    
    rand = Randomiser(df, name="R1")
    rand2 = Randomiser(df, name="R2")
    s = Search([rand, rand2])
    return image_handler, df, s

im, dmg, dmg_search = init_DMG()

100%|████████████████████████████████████████████████████████████████████████| 24824/24824 [00:27<00:00, 898.08it/s]


In [49]:
dmg.image_path.isna().sum()

24824

In [53]:
dmg["objectname_label"]

object_number
1992-0004_0-2       [sokkel, buste]
1992-0004_1-2               [buste]
1992-0004_2-2              [sokkel]
4521             [interieurweefsel]
5051                       [kussen]
                        ...        
2022-0028                    [kruk]
2022-0025_0-3                 [kom]
2022-0025_1-3                 [kom]
2022-0025_2-3                 [kom]
2022-0025_3-3                 [kom]
Name: objectname_label, Length: 24824, dtype: object

---
# DEV FUNCTIONS

In [40]:
def parse_id_list(id_list_str):
    try:
        return list(map(str.strip, id_list_str.split(",")))
    except ValueError:
        raise s

In [41]:
def search_collection(collection_id, object_ids, concept, model_list):
    cur_coll = get_collection(collection_id)
    object_ids = parse_id_list(object_ids)
    model_list = parse_id_list(model_list)
    cur_search = searches[collection_id]
    print(object_ids)
    scores = cur_search(object_ids)


object_ids = "1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"
object_ids = parse_id_list(object_ids)
object_ids
recs = dmg.loc[object_ids]

In [42]:
# searcher_scores = [cur_s(recs) for cur_s in dmg_search.searchers]
# searcher_scores = pd.DataFrame({cur_s.name: cur_s for cur_s in searcher_scores})
# searcher_scores.loc[recs.index] = 0.


# searcher_scores.round(3)

,R1,R2
object_number,,
1992-0004_0-2,0.716,0.848
1992-0004_1-2,0.255,0.244
1992-0004_2-2,0.202,0.023
4521,0.901,0.840
5051,0.388,0.050
...,...,...
2022-0028,0.184,0.891
2022-0025_0-3,0.841,0.669
2022-0025_1-3,0.658,0.908


---
# TESTING ROUTES

In [28]:
!curl "http://0.0.0.0:8080/collections"

[{"id":"DMG_2025-05-06","name":"Design Museum Gent (public & private)"}]

In [29]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06"

{"name":"Design Museum Gent (public & private)","id_":"DMG_2025-05-06","creation_timestamp":"2025-05-06","number_of_records":24824}

In [30]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/object-details?object_ids=1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"

[{"inventory_number":"1987-0120_04-14","title":"Ijslepeltje","description":"","designer":"","producer":"J.M. van Kempen en Zonen","design_date":"","production_date":"na 1907","design_place":"","production_place":"Nederland","rights_attribution":"In Copyright"},{"inventory_number":"1987-1343_2-4","title":"","description":"","designer":"","producer":"Alice Van Mourik","design_date":"","production_date":"","design_place":"","production_place":"","rights_attribution":"In Copyright"},{"inventory_number":"3370","title":"","description":"","designer":"","producer":"","design_date":"","production_date":"","design_place":"","production_place":"","rights_attribution":"In Copyright"},{"inventory_number":"3703_0-2","title":"","description":"","designer":"","producer":"","design_date":"","production_date":"","design_place":"","production_place":"","rights_attribution":"In Copyright"}]

In [31]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/models"

[{"id":139939504159680,"name":"Randomiser"}]

In [32]:
!curl "http://0.0.0.0:8080/moon?ISO_8601_datetime=2024-05-13&lat_degrees=51.05&long_degrees=3.71"

0.15026621349345307

In [33]:
# default params for search
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search"

{"1992-0004_0-2":0.1111888038613783,"1992-0004_1-2":0.9845061192577939,"1992-0004_2-2":0.19681230642108483,"4521":0.43159286325233515,"5051":0.4759896170957618,"5052":0.29484538552644535,"5055":0.48137968042379375,"5057_1-2":0.7593439738392277,"5057_2-2":0.8245467219252145,"5056":0.025658460278059736,"0901":0.506369841401871,"0977":0.6493101386614313,"0980":0.37793380472592075,"0008":0.8570449710676072,"0832":0.6711746504089309,"0833":0.019040735267610076,"0834":0.8193635625579113,"0836":0.112385622564524,"0843":0.738682456279943,"0844":0.41848120345099793,"0847":0.6937434297401621,"0848":0.07344683653988193,"0849":0.5794631489052763,"0850":0.5987341771866459,"0855":0.3660446798820072,"0856":0.3393266562826063,"0864":0.8335767870977722,"0867":0.23003808947978488,"0870":0.6725321373476616,"0872":0.7544620742320913,"0873":0.7136234553917142,"0875":0.3497645054425831,"0876":0.43835033551518177,"0879":0.4709577201889903,"0880":0.8798615326577283,"0881":0.6204257051366941,"0892":0.130656853

In [34]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b""

{"1992-0004_0-2":0.9754436230687917,"1992-0004_1-2":0.6385040025966061,"1992-0004_2-2":0.09650695986460744,"4521":0.42886441969447175,"5051":0.2471961352689378,"5052":0.05121517317416324,"5055":0.14052484342707028,"5057_1-2":0.4049223994417799,"5057_2-2":0.4685836733901405,"5056":0.933435275963074,"0901":0.09053386031697574,"0977":0.880777052691015,"0980":0.5274887291563591,"0008":0.3304716591730209,"0832":0.9865456593549908,"0833":0.022238280713960834,"0834":0.7020381913403102,"0836":0.33393763995700154,"0843":0.20703861836153215,"0844":0.3650261310130468,"0847":0.4174234280186876,"0848":0.22606710218228665,"0849":0.7085240050060543,"0850":0.4151340413876051,"0855":0.9560264160972801,"0856":0.3633820530574601,"0864":0.5261309972754029,"0867":0.9562492789769006,"0870":0.2882614513795315,"0872":0.045309106222688245,"0873":0.16973511800689778,"0875":0.3549571709338978,"0876":0.05575769430932764,"0879":0.20458006034857035,"0880":0.7172316422371032,"0881":0.42446004844687724,"0892":0.07142

In [35]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/order?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&skip=200&limit=100&reverse=true"

[{"inventory_number":"2013-0022","title":"Freeze-N-Stor","description":"Deze stapelbare diepvriesdoos dient om ijs of restjes van maaltijden in een diepvries te bewaren. Ze is gemaakt uit polyethyleen, een lichte, sterke en isolerende kunststof. Tupperware, opgericht in 1946 door Earl Tupper, werd beroemd met de massaproductie van kunststof huishoudproducten, in het bij …","designer":"Tupperware","producer":"","design_date":"ca. 1980 — 1980","production_date":"na ca. 1980","design_place":"Verenigde Staten","production_place":"Aalst","rights_attribution":"In Copyright"},{"inventory_number":"2721","title":"Strook Oud-Vlaamse kloskant van vlasgaren uit de Z …","description":"Vlas. Motieven in linnenslag verbonden door getande spijlen.\r\nStrook met rechte randen; geklost door getande spijlen. Golvende en krullende ranken met gestileerde bloemen of knoppen.\r\nVoetje in torchon kloskant.","designer":"","producer":"onbekend","design_date":"","production_date":"ca. 1681 — ca. 1700","design_p

In [36]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/sample?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&k=100"

[{"inventory_number":"2011-0008","title":"Shoggi","description":"De kruk 'Shoggi' is geïnspireerd op de traditionele zithouding van judoka op een tatami, een Japanse vloermat. Het kleine, ergonomische meubel dwingt de gebruiker tot een zithouding die gunstig is voor de rug en de bloedsomloop in de benen. De kruk is in de eerste plaats geschikt voor yoga- en medit …","designer":"Alain Gaubert","producer":"Atelier Gaubert","design_date":"1983 — 1983","production_date":"","design_place":"België","production_place":"Mondoubleau","rights_attribution":"In Copyright"},{"inventory_number":"1995-0039_9-9","title":"Bouquet","description":"","designer":"Floris Meydam","producer":"Glasfabriek Leerdam","design_date":"na 1986","production_date":"","design_place":"","production_place":"","rights_attribution":"In Copyright"},{"inventory_number":"1997-0059_01-11","title":"","description":"","designer":"Hartmut Wiese","producer":"FSB Franz Schneider Brakel","design_date":"na 1992","production_date":"na 

In [57]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/order/filter?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&filter_text="doos""

Internal Server Error